In [2]:
# All imports
import carla
import time
import sys


In [3]:
# Connect to CARLA
client = carla.Client('localhost', 2000)
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()


In [21]:
# Destroy all vehicles and sensors
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


In [15]:
# Ask user for a spawn index
spawn_index = int(input(f"Enter a spawn point index (0 to {len(spawn_points)-1}): "))

# Validate index
if 0 <= spawn_index < len(spawn_points):
    sp = spawn_points[spawn_index]
    world.debug.draw_string(sp.location, f"#{spawn_index}", draw_shadow=False,
                             color=carla.Color(r=0, g=255, b=0), life_time=60.0, persistent_lines=True)
    world.debug.draw_arrow(sp.location, sp.location + carla.Location(z=2),
                            thickness=0.1, arrow_size=0.3, color=carla.Color(r=0, g=255, b=0),
                            life_time=60.0, persistent_lines=True)
    print(f"Spawn point #{spawn_index} drawn.")
else:
    print("Invalid index.")



Spawn point #0 drawn.


In [16]:
# Get blueprints
blueprints = world.get_blueprint_library()
truck_bp = blueprints.filter('*firetruck*')[0]
mini_bp = blueprints.filter('*cooper_s*')[0]
vw_bp = blueprints.filter('*volkswagen*')[0]



In [17]:
#  Set spectator view
spectator = world.get_spectator()
spectator_pos = carla.Transform(start_point.location + carla.Location(x=20, y=10, z=4),
                                carla.Rotation(yaw=start_point.rotation.yaw - 155))
spectator.set_transform(spectator_pos)


In [27]:
# Spawn firetruck at first spawn point
start_point = spawn_points[0]
truck = world.try_spawn_actor(truck_bp, start_point)

if truck is not None:
    print("Firetruck spawned.")
    truck.set_autopilot(True)
    print("Truck initial position:", truck.get_transform())
else:
    print("Failed to spawn firetruck.")


Firetruck spawned.
Truck initial position: Transform(Location(x=0.000000, y=0.000000, z=0.000000), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))


In [31]:
# get actual position from the firetruck
truck_pos = truck.get_transform()
print(truck_pos)

Transform(Location(x=-114.535583, y=74.058922, z=-0.003508), Rotation(pitch=0.005136, yaw=90.055283, roll=0.048968))


In [32]:
#  Get map and roads
town_map = world.get_map()
print("Map loaded:", town_map.name)

roads = town_map.get_topology()
print("Number of road segments:", len(roads))


Map loaded: Carla/Maps/Town10HD_Opt
Number of road segments: 200


In [33]:
#  Print one waypoint
if roads:
    print("Sample waypoint:", roads[0][0])


Sample waypoint: Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))


In [34]:
#  Load route planner
sys.path.append('D:/carla/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner


In [ ]:
# Define start and end positions (as comments only)
# pos1 = Transform(Location(x=0.000000, y=0.000000, z=0.000000), Rotation(pitch=0.000007, yaw=0.318098, roll=0.000000))
# pos2 = Transform(Location(x=-0.989548, y=28.180809, z=-0.003467), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [46]:
from agents.navigation.global_route_planner import GlobalRoutePlanner

sampling_resolution = 2.0
grp = GlobalRoutePlanner(town_map, sampling_resolution)

# Define start and end
point_a = carla.Location(x=50.477512, y=141.135620, z=0.001844)
point_b = carla.Location(x=-64.644844, y=24.471010, z=0.600000)

# Trace and draw route
route = grp.trace_route(point_a, point_b)

for waypoint, _ in route:
    world.debug.draw_string(waypoint.transform.location, '^', draw_shadow=False,
                            color=carla.Color(r=0, g=0, b=255), life_time=120.0,
                            persistent_lines=True)
    world.debug.draw_arrow(waypoint.transform.location,
                           waypoint.transform.location + carla.Location(z=0.5),
                           thickness=0.1, arrow_size=0.3,
                           color=carla.Color(r=0, g=0, b=255), life_time=120.0,
                           persistent_lines=True)


In [ ]:
#  Show all available blueprints
print("All vehicle blueprints:")
for bp in blueprints.filter('vehicle.*'):
    print(bp.id)
